In [1]:
import networkx as nx
import obonet
import torch
import pandas as pd

/tmp/ipykernel_89641/2362945497.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
GO_graph = obonet.read_obo("GNN/go-basic.obo")

go_edges = []
for u, v, data in GO_graph.edges(data=True):
    go_edges.append([u, v])
go_edges_df = pd.DataFrame(go_edges, columns=['Source', 'Target']).dropna()
print(go_edges_df)
col_name = ['GO']
for i in range(1,769):
  col_name.append('feature'+str(i))
go_features_df = pd.read_csv("GNN/go_terms_embeddings.csv", skiprows=1, names=col_name).dropna()
print(go_features_df.head())

           Source      Target
0      GO:0000001  GO:0048308
1      GO:0000001  GO:0048311
2      GO:0000002  GO:0007005
3      GO:0000003  GO:0008150
4      GO:0000006  GO:0005385
...           ...         ...
83792  GO:2001317  GO:0034309
83793  GO:2001317  GO:0042181
83794  GO:2001317  GO:0120255
83795  GO:2001317  GO:1901362
83796  GO:2001317  GO:2001316

[83797 rows x 2 columns]
           GO  feature1  feature2  feature3  feature4  feature5  feature6  \
0  GO:0000001 -1.168093 -0.355214  0.265877 -0.710051  0.515028 -0.525165   
1  GO:0000002 -1.185879 -0.098765  0.388240 -0.295556  0.327296 -0.119842   
2  GO:0000003  0.063323 -0.199995  0.151511 -0.942141  0.109313  0.015316   
3  GO:0000005  0.163135  0.301527  0.219680  0.094342 -0.129769  0.225696   
4  GO:0000006 -0.641113 -0.541363  0.413941  0.699345  0.461507 -0.497388   

   feature7  feature8  feature9  ...  feature759  feature760  feature761  \
0 -0.186588 -0.161192  0.186984  ...   -1.350874   -0.991801   -0.648123   

In [3]:
col_name = ['protein']
for i in range(1,769):
  col_name.append('feature'+str(i))
gene_features_df = pd.read_csv('GNN/gene_embedding_GeneLLM_2.csv', header=None, names=col_name).dropna()
print(gene_features_df.head())

col_name = ['Target', 'Source']
go_protein_df = pd.read_csv(
    "GNN/mart_export.txt", 
    skiprows=1, 
    names=col_name, 
    usecols=[1, 2]  # 使用列的索引来指定
).dropna()
print(go_protein_df)

  protein  feature1  feature2  feature3  feature4  feature5  feature6  \
0     FES  0.339602 -0.030744 -0.901381  0.100888  0.886443  0.383596   
1  HADHA  -0.131799 -0.025745 -0.677301 -0.053545  0.971046  0.180315   
2  SLC7A7  0.385693 -0.070692 -0.847796 -0.022054  0.959772  0.085487   
3    LCK   0.650428  0.014479 -0.866163  0.053508  0.951529  0.269402   
4   HSPA2  0.322262  0.017484 -0.849302  0.046401  0.920429  0.463832   

   feature7  feature8  feature9  ...  feature759  feature760  feature761  \
0 -0.192082 -0.032063 -0.154869  ...   -0.549204   -0.856123    0.714672   
1 -0.028189 -0.077389 -0.095152  ...    0.927885   -0.817812    0.809631   
2  0.076455 -0.003006 -0.032268  ...    0.941094   -0.912443    0.789828   
3 -0.214788  0.045179 -0.506429  ...   -0.576739   -0.969558    0.916549   
4 -0.050414 -0.033398  0.387791  ...    0.387301   -0.860696    0.678607   

   feature762  feature763  feature764  feature765  feature766  feature767  \
0   -0.046649   -0.894424  

In [4]:
print(len(go_features_df))
go_features_df.rename(columns={'GO': 'protein'}, inplace=True)
combined_features = pd.concat([gene_features_df, go_features_df])
combined_features

47595


,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,FES,0.339602,-0.030744,-0.901381,0.100888,0.886443,0.383596,-0.192082,-0.032063,-0.154869,...,-0.549204,-0.856123,0.714672,-0.046649,-0.894424,-0.001815,0.739485,0.015581,-0.023863,-0.022002
1,HADHA,-0.131799,-0.025745,-0.677301,-0.053545,0.971046,0.180315,-0.028189,-0.077389,-0.095152,...,0.927885,-0.817812,0.809631,-0.005827,-0.848839,0.024516,0.526404,-0.039926,-0.102787,-0.026980
2,SLC7A7,0.385693,-0.070692,-0.847796,-0.022054,0.959772,0.085487,0.076455,-0.003006,-0.032268,...,0.941094,-0.912443,0.789828,0.046979,-0.715636,0.085842,0.150494,0.025392,-0.066035,-0.028283
3,LCK,0.650428,0.014479,-0.866163,0.053508,0.951529,0.269402,-0.214788,0.045179,-0.506429,...,-0.576739,-0.969558,0.916549,-0.080332,-0.927649,-0.047398,0.741663,-0.000096,-0.096318,-0.056501
4,HSPA2,0.322262,0.017484,-0.849302,0.046401,0.920429,0.463832,-0.050414,-0.033398,0.387791,...,0.387301,-0.860696,0.678607,-0.060695,-0.945793,0.040472,0.831079,-0.001711,-0.079842,-0.011189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47590,GO:2001313,0.174428,0.194728,-0.284376,0.282102,-0.713190,-0.272055,0.121190,0.129901,-0.983496,...,0.500545,0.429651,-0.292929,-0.464941,-0.740187,0.179149,-0.960807,-0.746958,1.069112,-0.848182
47591,GO:2001314,0.025886,0.306214,-0.254303,0.253673,-0.533680,-0.269355,0.150939,-0.229323,-1.078991,...,0.042979,0.134560,-0.356661,-0.381828,-0.638338,0.077176,-0.788312,-0.683442,1.087031,-0.593092
47592,GO:2001315,0.027134,0.241391,-0.227353,0.317366,-0.726657,-0.197968,0.045653,0.038912,-0.954113,...,0.349853,0.370059,-0.144606,-0.493184,-0.655063,0.217335,-0.841272,-0.821077,1.036363,-0.836614
47593,GO:2001316,0.139543,0.028883,0.899480,0.152932,0.576852,0.330342,0.916943,0.012306,-0.020316,...,-0.354748,-0.083168,0.043640,-0.663565,0.543016,-0.652230,-1.427882,-0.985257,1.673561,0.109659


In [6]:
import numpy as np
nodes = combined_features['protein']

# 定义嵌入的维度
embedding_dim = 768

# 生成随机嵌入
node_embeddings = {node: np.random.rand(embedding_dim) for node in nodes}

# 创建列表，每个元素为一个包含节点名和嵌入的字典
data = [{'Node': node, 'Embedding': embedding} for node, embedding in node_embeddings.items()]

# 转换为DataFrame
df = pd.DataFrame(data)

# 查看DataFrame的前几行
print(df)

             Node                                          Embedding
0             FES  [0.4438934076953207, 0.6547635781497536, 0.479...
1          HADHA   [0.43062846341232963, 0.17973569083045415, 0.6...
2          SLC7A7  [0.5660207802974483, 0.9893016741119274, 0.889...
3            LCK   [0.05244531785542528, 0.7042113750619634, 0.44...
4           HSPA2  [0.06608896989532431, 0.6718893790647981, 0.45...
...           ...                                                ...
62040  GO:2001313  [0.76197988097058, 0.6578877828420915, 0.55918...
62041  GO:2001314  [0.14085847914459715, 0.7352766352762377, 0.59...
62042  GO:2001315  [0.829709428784575, 0.4123918009929196, 0.5760...
62043  GO:2001316  [0.7240967613247861, 0.5148399788025295, 0.747...
62044  GO:2001317  [0.23188587195825383, 0.6366218499635815, 0.99...

[62045 rows x 2 columns]


In [7]:
features_tensor = torch.tensor(df['Embedding'].tolist(), dtype=torch.float)
features_tensor

/tmp/ipykernel_89641/2498144971.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  features_tensor = torch.tensor(df['Embedding'].tolist(), dtype=torch.float)


tensor([[0.4439, 0.6548, 0.4793,  ..., 0.8343, 0.8348, 0.1999],
        [0.4306, 0.1797, 0.6233,  ..., 0.3515, 0.3180, 0.4356],
        [0.5660, 0.9893, 0.8894,  ..., 0.5455, 0.6846, 0.7396],
        ...,
        [0.8297, 0.4124, 0.5761,  ..., 0.9669, 0.6896, 0.4834],
        [0.7241, 0.5148, 0.7475,  ..., 0.6062, 0.8720, 0.7089],
        [0.2319, 0.6366, 0.9992,  ..., 0.6943, 0.1489, 0.1729]])

In [8]:
gene_edges_df = pd.read_csv('GNN/protein_interactions.csv', usecols=[0, 1], names=col_name).dropna()

In [9]:
combined_edges = pd.concat([go_protein_df, go_edges_df, gene_edges_df])
#combined_edges = combined_edges[['Source', 'Target']]
combined_edges

,Target,Source
0,MT-TF,GO:0030533
1,MT-TF,GO:0006412
4,MT-RNR2,GO:0003735
5,MT-RNR2,GO:0005840
6,MT-TL1,GO:0030533
...,...,...
13715124,LDB1,SAMD14
13715125,LDB1,KDM6B
13715126,LDB1,WWP2
13715127,LDB1,VPS33B


In [10]:
nodes_in_features = set(combined_features['protein'])

filtered_edges_df = combined_edges[
    combined_edges['Source'].isin(nodes_in_features) & combined_edges['Target'].isin(nodes_in_features)
]


In [11]:
node_id_to_index = {node_id: i for i, node_id in enumerate(combined_features['protein'])}
# 确保edge_index是按照这个新的索引顺序排列的
source_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['Source']]
target_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['Target']]
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)
edge_index

tensor([[24323, 17460, 27132,  ...,   947,  6874, 13222],
        [ 2077,  2077,  2077,  ..., 10107, 10107, 10107]])

In [12]:
labels_df = pd.read_csv('GNN/new_labels.csv')
labels_df.rename(columns={'Gene name': 'protein'}, inplace=True)
labels_df.dropna()

,Unnamed: 0,protein,Solubility,Label,Word_Count,Count_Category
0,0,ERAP2,Membrane,0,117,0
1,1,ADAMTSL5,Soluble,1,28,1
2,2,TBC1D30,Membrane,0,55,0
3,3,KCNK18,Membrane,0,184,0
4,4,NDNF,Soluble,1,129,0
...,...,...,...,...,...,...
1374,1374,TRABD2B,Membrane,0,96,0
1375,1375,RPS9,Soluble,1,205,0
1376,1376,SLC22A16,Membrane,0,93,0
1377,1377,FBN3,Soluble,1,90,0


In [13]:
label_indices = [node_id_to_index[node_id] for node_id in labels_df['protein']]
print(label_indices)
num_nodes = len(combined_features)
labels = torch.full((num_nodes,), -1, dtype=torch.long)
for i, index in enumerate(labels_df['Label']):
    labels[label_indices[i]] = index

labels_tensor = torch.tensor(labels, dtype=torch.long)

[67, 81, 93, 94, 100, 116, 129, 142, 148, 149, 157, 167, 170, 202, 228, 236, 241, 243, 255, 258, 276, 281, 287, 294, 305, 319, 321, 334, 340, 369, 373, 375, 383, 384, 399, 402, 418, 447, 455, 474, 483, 484, 490, 498, 511, 518, 526, 558, 569, 585, 586, 587, 593, 594, 595, 596, 599, 615, 641, 653, 654, 664, 682, 736, 754, 762, 764, 773, 777, 781, 833, 852, 894, 895, 899, 915, 917, 938, 944, 957, 982, 983, 997, 1003, 1019, 1040, 1047, 1053, 1062, 1064, 1065, 1067, 1074, 1075, 1078, 1079, 1105, 1108, 1115, 1116, 1120, 1124, 1131, 1136, 1137, 1143, 1157, 1170, 1173, 1183, 1195, 1196, 1197, 1215, 1235, 1236, 1237, 1243, 1256, 1261, 1262, 1273, 1274, 1277, 1311, 1315, 1327, 1333, 1347, 1363, 1371, 1377, 1378, 1383, 1392, 1397, 1426, 1427, 1431, 1435, 1444, 1445, 1469, 1470, 1475, 1484, 1489, 1492, 1493, 1521, 1527, 1540, 1548, 1559, 1575, 1593, 1606, 1609, 1614, 1617, 1619, 1637, 1638, 1645, 1674, 1681, 1710, 1720, 1733, 1736, 1737, 1759, 1776, 1795, 1807, 1823, 1833, 1834, 1844, 1848, 1849, 

/tmp/ipykernel_89641/2534729209.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)


In [14]:
from torch_geometric.data import Data
data = Data(x=features_tensor, edge_index=edge_index, y=labels_tensor)

print("x:", data.x.shape, data.x.dtype)
print("edge_index:", data.edge_index.shape, data.edge_index.dtype)
print("labels:", data.y.shape, data.y.dtype)

x: torch.Size([62045, 768]) torch.float32
edge_index: torch.Size([2, 9914754]) torch.int64
labels: torch.Size([62045]) torch.int64


In [15]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear, ModuleList, Dropout

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_layers, activation, dropout):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.convs = ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 2)])
        self.conv_last = GCNConv(hidden_dim, num_classes)
        self.activation = activation
        self.dropout = Dropout(dropout)
        self.num_layers = num_layers

    def forward(self, x, edge_index):
        # 输入层
        x = self.conv1(x, edge_index)
        x = self.activation(x)
        x = self.dropout(x)
        
        # 隐藏层
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)
            x = self.dropout(x)

        # 输出层
        x = self.conv_last(x, edge_index)
        return x

In [16]:
from __future__ import division

import torch


def accuracy(pred, target):
    r"""Computes the accuracy of correct predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.

    :rtype: int
    """
    return (pred == target).sum().item() / target.numel()



def true_positive(pred, target, num_classes):
    r"""Computes the number of true positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target == i)).sum())

    return torch.tensor(out)



def true_negative(pred, target, num_classes):
    r"""Computes the number of true negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target != i)).sum())

    return torch.tensor(out)



def false_positive(pred, target, num_classes):
    r"""Computes the number of false positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target != i)).sum())

    return torch.tensor(out)



def false_negative(pred, target, num_classes):
    r"""Computes the number of false negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target == i)).sum())

    return torch.tensor(out)



def precision(pred, target, num_classes):
    r"""Computes the precision:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FP}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fp = false_positive(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fp)
    out[torch.isnan(out)] = 0

    return out



def recall(pred, target, num_classes):
    r"""Computes the recall:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FN}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fn = false_negative(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fn)
    out[torch.isnan(out)] = 0

    return out



def f1_score(pred, target, num_classes):
    r"""Computes the :math:`F_1` score:
    :math:`2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}
    {\mathrm{precision}+\mathrm{recall}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    prec = precision(pred, target, num_classes)
    rec = recall(pred, target, num_classes)

    score = 2 * (prec * rec) / (prec + rec)
    score[torch.isnan(score)] = 0

    return score

In [17]:
from sklearn.preprocessing import label_binarize
def train_model_scheduler(model, masked_features, labels, edge_index, optimizer, criterion, scheduler, train_mask):
    model.train()  # 设置模型为训练模
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index)  # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    scheduler.step(loss)
    return loss.item()

def train_model(model, masked_features, labels, edge_index, optimizer, criterion, train_mask):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index) # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

def evaluate_model(model, features, labels, edge_index, mask):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        # 获取模型输出，这里假设输出已经是经过sigmoid的概率
        probabilities = model(features, edge_index)
        if probabilities.shape[1] == 2:  # 假设有两个输出（每个类一个概率）
            positive_probs = probabilities[mask, 1]  # 选择正类概率
        else:
            positive_probs = probabilities[mask]  # 如果只有一个输出，假设已经是正类概率
        val_f1 = torch.mean(f1_score(torch.argmax(probabilities[mask],dim=1), labels[mask], num_classes=2)).cpu().numpy()
        auc_score = roc_auc_score(labels[mask].cpu().numpy(), positive_probs.cpu().numpy())

    return val_f1, auc_score

In [18]:
from sklearn.metrics import roc_auc_score
import numpy as np
import random
from sklearn.model_selection import train_test_split
# 实例化模型
device = torch.device('cpu')
data = data.to(device)
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)


labeled_indices = label_indices
random.shuffle(labeled_indices)
num_labeled = len(labeled_indices)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

276
tensor([False, False, False,  ..., False, False, False])


In [19]:
num_epochs = 1000
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_mask)
    
    if (epoch + 1) % 50 == 0: 
        print(f'Epoch {epoch}: Train Loss:{train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

Epoch 49: Train Loss:0.9474, Macro_F1: 0.3429, AUC_score: 0.5231
Epoch 99: Train Loss:0.6797, Macro_F1: 0.3371, AUC_score: 0.5276
Epoch 149: Train Loss:0.6938, Macro_F1: 0.3235, AUC_score: 0.5327
Epoch 199: Train Loss:0.6827, Macro_F1: 0.5208, AUC_score: 0.5305
Epoch 00249: reducing learning rate of group 0 to 2.0000e-03.
Epoch 249: Train Loss:0.6716, Macro_F1: 0.6143, AUC_score: 0.5507
Epoch 299: Train Loss:0.6754, Macro_F1: 0.6123, AUC_score: 0.5558
Epoch 349: Train Loss:0.6704, Macro_F1: 0.5540, AUC_score: 0.5537
Epoch 399: Train Loss:0.6839, Macro_F1: 0.5817, AUC_score: 0.5654
Epoch 00418: reducing learning rate of group 0 to 4.0000e-04.
Epoch 449: Train Loss:0.6792, Macro_F1: 0.6050, AUC_score: 0.5691
Epoch 499: Train Loss:0.6669, Macro_F1: 0.5987, AUC_score: 0.5692
Epoch 549: Train Loss:0.6670, Macro_F1: 0.6114, AUC_score: 0.5704
Epoch 599: Train Loss:0.6701, Macro_F1: 0.5817, AUC_score: 0.5802
Epoch 00605: reducing learning rate of group 0 to 8.0000e-05.
Epoch 649: Train Loss:0.